In [ ]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
# Готовые спарсенные данные
df = pickle.load(open('pickle_data_test', 'rb'))

In [ ]:
# делим на две выборки
df_test =df[(df.gender == '-') & (df.age == '-')]
df_train = df[(df.gender != '-') & (df.age != '-')]
len(df_train), len(df_test)

In [ ]:
# выделим таргеты
gender = df_train['gender']
age = df_train['age']
df_train['ga'] = df_train['gender'] + df_train['age']

In [ ]:
# уберем ненужные ячейки
df_train= df_train[['uid','ga','all_words']]

In [ ]:
# сделаем общий таргет через ohe
df_train = pd.get_dummies(df_train, columns = ['ga']) 
target = df_train.drop(['uid', 'all_words'], axis=1)


In [ ]:
# токенизатор
def tokenize(text):
    tokens = text.split(';') #так у меня разные источники объеденялись
    res = []
    for i in tokens:
        if i.strip():
            for j in i.split(' '):
                if j.strip(): res.append(j.strip().lower())  
    return res

In [ ]:
# делаем большму матрицу tf idf
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(norm=None, tokenizer = tokenize, min_df=7)
matrix = tfidf.fit_transform(df_train['all_words']).toarray()
matrix.shape

In [ ]:
#добавим слова в датафрейм
words = [x[0] for x in sorted(tfidf.vocabulary_.items(), key=lambda x: x[1])]
matrix = pd.DataFrame(matrix, columns=words)

In [ ]:
# матрица для предсказания
matrix_res = tfidf.transform(df_test['all_words']).toarray()
matrix_res = pd.DataFrame(matrix_res, columns=words)
matrix_res.shape

In [ ]:
# ещё данные
df1 = pickle.load(open('pickle_data_project', 'rb'))

In [ ]:
# прицепим их
matrix = pd.concat([matrix, df1_train], axis=1)
matrix.shape

In [ ]:
matrix_res = pd.concat([matrix_res, df1_test], axis=1)
matrix_res.shape

In [ ]:
# сохранимся
import pickle
pickle.dump(matrix, open('pickle_data_matrix', 'wb'), protocol=4)
pickle.dump(matrix_res, open('pickle_data_matrix_res', 'wb'), protocol=4)

In [ ]:
matrix = pickle.load(open('pickle_data_matrix', 'rb'))
matrix_res = pickle.load(open('pickle_data_matrix_res', 'rb'))

In [ ]:
#пробуем рандомфорест
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 700, verbose=1,n_jobs=4)
forest.fit(matrix, target)

In [ ]:
# сохранимся
import pickle
pickle.dump(forest, open('pickle_gb_ft', 'wb'), protocol=4)

In [ ]:
result = forest.predict(matrix_res)
result_proba = forest.predict_proba(matrix_res)

# дальше был вывод модели для финального файла. Но там были плохие результаты

# feature importance

In [ ]:
# посмотрим веса
features_weights = np.array([i.feature_importances_ for i in forest.estimators_]).sum(axis=0)

In [ ]:
# оставим все что важно
features = pd.DataFrame(features_weights, index=matrix.columns, columns=['importance'])
features.sort_values(by='importance', ascending=False, inplace=True)
features_s = features[features['importance']>0]
len(features_weights), len(features_s)

In [ ]:
matrix_cut = matrix[features_s.index.tolist()]
matrix_cut.shape

In [ ]:
matrix_cut_res = matrix_res[features_s.index.tolist()]
matrix_cut_res.shape

In [ ]:
# сохранимся. Главное вовремя сохраниться.
import pickle
pickle.dump(matrix_cut, open('pickle_matrix_cut', 'wb'), protocol=4)
pickle.dump(matrix_cut_res, open('pickle_matrix_cut_res', 'wb'), protocol=4)
pickle.dump(target, open('pickle_matrix_target', 'wb'),protocol=4)

In [ ]:
import pickle
matrix_cut = pickle.load(open('pickle_matrix_cut', 'rb'))
matrix_cut_res = pickle.load(open('pickle_matrix_cut_res', 'rb'))
target = pickle.load(open('pickle_matrix_target', 'rb'))

In [ ]:
# градиент буст на собранном таргете. Работал лучше рандом фореста
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multiclass import OneVsRestClassifier
gb = OneVsRestClassifier(GradientBoostingClassifier(n_estimators = 800, verbose=1,), n_jobs=5, ) 

gb.fit(matrix_cut, target)

In [ ]:
import pickle
pickle.dump(gb, open('pickle_gb', 'wb'), protocol=4)

# Devide gender and age

In [ ]:
import pandas as pd

In [ ]:
import pickle
matrix_cut = pickle.load(open('pickle_matrix_cut', 'rb'))
matrix_cut_res = pickle.load(open('pickle_matrix_cut_res', 'rb'))
target = pickle.load(open('pickle_matrix_target', 'rb'))

In [ ]:
# обучимся по полу. OVR лишний конечно же.
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multiclass import OneVsRestClassifier
ovr1 = OneVsRestClassifier(GradientBoostingClassifier(n_estimators = 800, verbose=1,), n_jobs=4, ) 

ovr1.fit(matrix_cut, gender_number)
import pickle
pickle.dump(ovr1, open('ovr1.pckl', 'wb'), protocol=4)

In [ ]:
# закодируем пол что бы потом по нему обучаться
gender_number = [1 if x == 'M' else 0 for x in gender]
matrix_cut['gender'] = gender_number

In [ ]:
# закодируем в ohe возраст
age_dummies = pd.get_dummies(age, columns = ['age'])

In [ ]:
# обучимся по возрасту, зная пол
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multiclass import OneVsRestClassifier
ovr2 = OneVsRestClassifier(GradientBoostingClassifier(n_estimators = 150, verbose=777, max_depth=4), n_jobs=5, ) 

ovr2.fit(matrix_cut, age_dummies)

import pickle
pickle.dump(ovr2, open('ovr2_with_gender2.pckl', 'wb'), protocol=4)

In [ ]:
# делаем предсказания
pred_gender = ovr1.predict(matrix_cut_res)
pred_gender_proba = ovr1.predict_proba(matrix_cut_res)
pred_gender_proba = [max(i) for i in pred_gender_proba]
matrix_cut_res['gender'] = pred_gender
pred_age = ovr2.predict_proba(matrix_cut_res)

In [ ]:
# от форматируем всё как было. Выберем самый вероятный возраст.
pred_age_done = [[max(row), age_dummies.columns[np.where(row==max(row))[0][0]]] for row in pred_age]
proba = [item[0] for item in pred_age_done]
age = [item[1] for item in pred_age_done]
gender = ['M' if x == 1 else 'F' for x in pred_gender]

In [ ]:
# соберем две вероятности в одну
proba_test = []
for i in range(len(proba)):
    proba_test.append((proba[i] ** 1.7) * pred_gender_proba[i])

In [ ]:
# выкладываем финальный датасет 
z = pd.DataFrame(df_test['uid'])
z['gender'] = gender
z['proba'] = proba_test
z['age'] =age
z.sort_values(by='proba', axis=0, ascending=False, inplace=True)
z.set_index('uid', inplace=True)
pickle.dump(z, open('pickle_final', 'wb'))